Use this dataset of airline arrival information to predict how late flights will be. A flight only counts as late if it is more than 30 minutes late.

In [26]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
sns.set_style('white')

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest

pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)

Variable descriptions - Name - Description
    
1 	Year 	1987-2008

2 	Month 	1-12

3 	DayofMonth 	1-31

4 	DayOfWeek 	1 (Monday) - 7 (Sunday)

5 	DepTime 	actual departure time (local, hhmm)

6 	CRSDepTime 	scheduled departure time (local, hhmm)

7 	ArrTime 	actual arrival time (local, hhmm)

8 	CRSArrTime 	scheduled arrival time (local, hhmm)

9 	UniqueCarrier 	unique carrier code

10 	FlightNum 	flight number

11 	TailNum 	plane tail number

12 	ActualElapsedTime 	in minutes

13 	CRSElapsedTime 	in minutes

14 	AirTime 	in minutes

15 	ArrDelay 	arrival delay, in minutes

16 	DepDelay 	departure delay, in minutes

17 	Origin 	origin IATA airport code

18 	Dest 	destination IATA airport code

19 	Distance 	in miles

20 	TaxiIn 	taxi in time, in minutes

21 	TaxiOut 	taxi out time in minutes

22 	Cancelled 	was the flight cancelled?

23 	CancellationCode 	reason for cancellation (A = carrier, B = weather, C = NAS, D = security)

24 	Diverted 	1 = yes, 0 = no

25 	CarrierDelay 	in minutes

26 	WeatherDelay 	in minutes

27 	NASDelay 	in minutes

28 	SecurityDelay 	in minutes

29 	LateAircraftDelay 	in minutes

In [2]:
import pandas as pd
df = pd.read_csv("1995.csv", encoding='latin1')

# Data Cleaning Section:

## Looks like all the types of delays and cancellation codes are useless. I need to drop them from the table.

In [4]:
df.drop(['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
        'LateAircraftDelay', 'CancellationCode'], 1, inplace=True)

In [5]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1995,1,6,5,657.0,645,952.0,937,UA,482,N7298U,115.0,112.0,83.0,15.0,12.0,ORD,PHL,678.0,7,25,0,0
1,1995,1,7,6,648.0,645,938.0,937,UA,482,N7449U,110.0,112.0,88.0,1.0,3.0,ORD,PHL,678.0,5,17,0,0
2,1995,1,8,7,649.0,645,932.0,937,UA,482,N7453U,103.0,112.0,83.0,-5.0,4.0,ORD,PHL,678.0,3,17,0,0
3,1995,1,9,1,645.0,645,928.0,937,UA,482,N7288U,103.0,112.0,84.0,-9.0,0.0,ORD,PHL,678.0,3,16,0,0
4,1995,1,10,2,645.0,645,931.0,937,UA,482,N7275U,106.0,112.0,82.0,-6.0,0.0,ORD,PHL,678.0,6,18,0,0


## We can also drop Year since they're all the same value. DayofMonth is random throughout the year so we can drop. Also, FlightNum is arbitrary whereas TailNum could signal a specific plane model or manufacturer. 

In [9]:
df.drop(['Year', 'FlightNum', 'DayofMonth'], 
        axis=1, inplace=True)

ValueError: labels ['Year' 'FlightNum' 'DayofMonth'] not contained in axis

## We can eliminate Diverted and Cancelled because those are already associated with delays. 

In [11]:
df.drop(['Cancelled', 'Diverted'], 
        axis=1, inplace=True)

## We can also drop DepTime and ActualElapsedTime for the same reason. We can recreate those features from the CRSDepTime and CRSElapsedTime against the delays.

In [13]:
df.drop(['DepTime', 'ActualElapsedTime'], 
        axis=1, inplace=True)

## We have reduced our dataset to the 14 most unique and essential pieces of information (whew). Now, we're on to the next logical step of preparing our data.

# Clearing NaN's

## Sampling the data shows we have NaN's in the delays

In [15]:
df.sample(50)

,Month,DayOfWeek,CRSDepTime,UniqueCarrier,TailNum,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
4908929,12,1,858,DL,N475DA,67.0,45.0,76.0,73.0,ATL,PNS,272.0,6,19
1642273,4,1,2144,TW,N978Z,81.0,60.0,3.0,1.0,STL,ICT,392.0,4,19
4010349,9,3,1332,AA,N402AA,103.0,77.0,65.0,74.0,ABQ,DFW,569.0,7,10
1123818,3,5,0,AA,N550AA,233.0,210.0,1.0,0.0,LGA,DFW,1389.0,12,12
758367,2,7,937,DL,N116DL,243.0,90.0,NaN,NaN,ATL,LAS,1747.0,9,11
4983051,12,6,1440,TW,N931TW,209.0,201.0,78.0,65.0,SAN,STL,1558.0,4,17
171759,1,1,1100,NW,N918RW,65.0,55.0,6.0,-3.0,OMA,MSP,282.0,5,14
3753551,9,6,1000,WN,N367,150.0,135.0,3.0,0.0,LAS,OMA,1099.0,3,15
3486112,8,4,550,DL,N916DE,104.0,82.0,-3.0,3.0,MIA,ATL,595.0,4,12
3514850,8,7,1025,TW,N916TW,162.0,137.0,32.0,14.0,STL,BOS,1046.0,7,36


## We have a count of 5,327,435 flights on record

In [17]:
df.count()

Month             5327435
DayOfWeek         5327435
CRSDepTime        5327435
UniqueCarrier     5327435
TailNum           5327435
CRSElapsedTime    5324186
AirTime           5277086
ArrDelay          5225038
DepDelay          5235530
Origin            5327435
Dest              5327435
Distance          5321448
TaxiIn            5327435
TaxiOut           5327435
dtype: int64

## Here is the breakdown of how many NaN values are in each column

In [16]:
df.isnull().sum()

Month                  0
DayOfWeek              0
CRSDepTime             0
UniqueCarrier          0
TailNum                0
CRSElapsedTime      3249
AirTime            50349
ArrDelay          102397
DepDelay           91905
Origin                 0
Dest                   0
Distance            5987
TaxiIn                 0
TaxiOut                0
dtype: int64

## If we drop all the rows with NaN we would only lose < 3% of our dataset. That's pretty good and will only have a negligible impact on our model.

In [18]:
df_size = df.shape[0]
no_nulls = df.dropna().shape[0]
print("Out of " + str(df_size) + " rows, " + str(df_size - no_nulls) + " will be lost by dropping NaN's")
remaining = no_nulls/df_size
print("" + str(remaining) + '% of the values will remain.')
df.dropna(inplace=True)

Out of 5327435 rows, 108295 will be lost by dropping NaN's
0.9796722062305782% of the values will remain.


# Data Cleaning part 2: Converting Time

## We have two problems in this next stage of data cleaning

## First, we need categorize the columns for UniqueCarrier, TailNum, Origin, and Dest numerically so Python can understand.

In [20]:
from sklearn.preprocessing import LabelEncoder
categ = ['UniqueCarrier', 'Origin', 'Dest', 'TailNum']
le = LabelEncoder()
for col in categ:
    df[col] = le.fit_transform(df[col])

In [21]:
df.head()

,Month,DayOfWeek,CRSDepTime,UniqueCarrier,TailNum,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,5,645,7,2523,112.0,83.0,15.0,12.0,156,164,678.0,7,25
1,1,6,645,7,2558,112.0,88.0,1.0,3.0,156,164,678.0,5,17
2,1,7,645,7,2563,112.0,83.0,-5.0,4.0,156,164,678.0,3,17
3,1,1,645,7,2508,112.0,84.0,-9.0,0.0,156,164,678.0,3,16
4,1,2,645,7,2486,112.0,82.0,-6.0,0.0,156,164,678.0,6,18


## Secondly, the times used in CRSDepTime are expressed in military time whereas the time values measuring how long the delays were and airtime spent are expressed in minutes. We need to make this consisent in minutes since all we have to do is change CRSDepTime.

In [22]:
def get_last_digits(num, digits=2):
    """
    This converts a time to purely minutes.
    IN: a 'time' in the form 1647 or 832; it's dealing with a bad format
    OUT: an integer
    EX: get_last_ditigs(1647) = 1007
        get_last_digits(632)  = 392
    """
    return (num // 100)*60 + (num % 10**digits)

In [23]:
df['CRSDepTime'] = df['CRSDepTime'].apply(get_last_digits)

In [24]:
df.head()

,Month,DayOfWeek,CRSDepTime,UniqueCarrier,TailNum,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,5,405,7,2523,112.0,83.0,15.0,12.0,156,164,678.0,7,25
1,1,6,405,7,2558,112.0,88.0,1.0,3.0,156,164,678.0,5,17
2,1,7,405,7,2563,112.0,83.0,-5.0,4.0,156,164,678.0,3,17
3,1,1,405,7,2508,112.0,84.0,-9.0,0.0,156,164,678.0,3,16
4,1,2,405,7,2486,112.0,82.0,-6.0,0.0,156,164,678.0,6,18


# Model Time !!

## Collecting my subsample

In [27]:

train_sample = df.sample(100000)
y = train_sample['ArrDelay']
X = train_sample.drop('ArrDelay', 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [28]:
rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [29]:
predictions = rfr.predict(X_test)
print('\nR-squared:')
print(rfr.score(X_test, y_test))
cross = cross_val_score(rfr, X_train, y_train, cv=5)
print(cross)
print("Cross Validation Mean: ", cross.mean())


R-squared:
0.950540471364
[ 0.82858427  0.95144794  0.94470823  0.95413958  0.94958169]
Cross Validation Mean:  0.925692338715


In [30]:
rfr.feature_importances_

array([ 0.00453625,  0.0023172 ,  0.00591117,  0.00422996,  0.00523005,
        0.03671227,  0.04568981,  0.80134427,  0.00451556,  0.00448675,
        0.00489473,  0.01308409,  0.06704791])

In [31]:
selector = SelectKBest(k=13)
X_new = selector.fit_transform(X_train, y_train)
names = X.columns.values[selector.get_support()]
scores = selector.scores_[selector.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)


        Feat_names    F_Scores
7         DepDelay  812.972703
12         TaxiOut   44.026687
5   CRSElapsedTime   14.250885
6          AirTime   11.405483
10        Distance   11.398679
11          TaxiIn    9.047958
2       CRSDepTime    3.936688
3    UniqueCarrier    3.217361
0            Month    1.458047
1        DayOfWeek    1.077699
9             Dest    1.033958
8           Origin    0.999761
4          TailNum    0.997042


In [193]:
#before
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5327435 entries, 0 to 5327434
Data columns (total 21 columns):
Month                int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             float64
TaxiIn               int64
TaxiOut              int64
Cancelled            int64
CancellationCode     float64
Diverted             int64
dtypes: float64(9), int64(8), object(4)
memory usage: 853.5+ MB


## There are still a lot of NaN values across the dataset. 